In [1]:
import pandas as pd
import numpy as np

import multiprocessing
import gc

from time import time
import datetime

from tqdm import tqdm_notebook

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.feature_selection import RFECV

import lightgbm as lgb

sns.set()
%matplotlib inline

In [2]:
%%time

base_path = 'C:/Users/VenD/Downloads/ieee-fraud-detection/dataset/'

train_transaction = pd.read_csv(base_path + 'train_transaction.csv')
train_identity = pd.read_csv(base_path + 'train_identity.csv')

test_identity = pd.read_csv(base_path + 'test_identity.csv')
test_transaction = pd.read_csv(base_path + 'test_transaction.csv')

sample_submission = pd.read_csv(base_path + 'sample_submission.csv')

train = train_transaction.merge(train_identity, how='left', on='TransactionID')
test = test_transaction.merge(test_identity, how='left', on='TransactionID')

del train_transaction, train_identity, test_transaction, test_identity
gc.collect()

Wall time: 55.1 s


In [9]:
# Function to reduce the DF size
def reduce_memory_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    df_memory_size_in_mega_bytes = df.memory_usage().sum() / 1024**2
    for col in tqdm_notebook(df.columns):
        col_data_type = df[col].dtypes
        if col_data_type in numerics:
            col_min = df[col].min()
            col_max = df[col].max()
            if str(col_data_type)[:3] == 'int':
                if col_min > np.iinfo(np.int8).min and col_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif col_min > np.iinfo(np.int16).min and col_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif col_min > np.iinfo(np.int32).min and col_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif col_min > np.iinfo(np.int64).min and col_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64) 
            else:
                if col_min > np.finfo(np.float16).min and col_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif col_min > np.finfo(np.float32).min and col_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                elif col_min > np.finfo(np.float64).min and col_max < np.finfo(np.float64).max:
                    df[col] = df[col].astype(np.float64)
                    
    reduced_df_memory_size_in_mega_bytes = df.memory_usage().sum() / 1024**2
    
    memory_diff = (df_memory_size_in_mega_bytes - reduced_df_memory_size_in_mega_bytes)
    reduce_percent =  (memory_diff / df_memory_size_in_mega_bytes) * 100
    
    if verbose:
        print('Memory usage decreased from {:5.2f} Mb to {:5.2f} Mb ({:.1f}% reduction)' \
              .format(df_memory_size_in_mega_bytes, reduced_df_memory_size_in_mega_bytes, reduce_percent))
        
    return df

In [10]:
train = reduce_memory_usage(train)
test = reduce_memory_usage(test)


Memory usage decreased from 1959.88 Mb to 650.48 Mb (66.8% reduction)



Memory usage decreased from 1677.73 Mb to 565.37 Mb (66.3% reduction)


In [4]:
def one_unique_value_features(df):
    one_value_cols = [col for col in df if df[col].nunique() == 1]
    
    return one_value_cols

In [5]:
def missing_value_features(df, threshold = 0.0):
    missing_value_cols = [col for col in df if (df[col].isnull().sum() / df.shape[0]) > threshold]
    
    return missing_value_cols

In [6]:
def frequent_value_features(df, threshold = 0.0):
    frequent_top_value_cols = [col for col in df if df[col].value_counts(dropna=False, normalize=True).values[0] > threshold]
    
    return frequent_top_value_cols

In [14]:
# Function to calculate missing values by column
def missing_values_table(df):
    # Total missing values
    miss_val = df.isnull().sum()
    
    # Percentage of missing values
    miss_val_percent = (miss_val / len(df)) * 100
    
    # Make a table with the results
    miss_val_table = pd.concat([miss_val, miss_val_percent], axis = 1)
    
    # Rename the columns
    miss_val_table_ren_cols = miss_val_table.rename(columns = {0: 'Missing Values', 1: '% of Total Values'})
    
    # Sort (descending) the table by percentage of missing
    sorted_miss_val_table = miss_val_table_ren_cols[miss_val_table_ren_cols.iloc[:, 0] != 0]. \
                            sort_values('Missing Values', ascending = False).round(1)
    
    # Print some summary information
    print("Your selected dataframe has {} columns.\nThere are {} columns that have missing values."
          .format(df.shape[1], miss_val_table_ren_cols.shape[0]))
    
    # Return the dataframe with missing information
    return sorted_miss_val_table

In [15]:
# Missing values statistics
missing_values = missing_values_table(train)
missing_values.head(20)

Your selected dataframe has 434 columns.
There are 434 columns that have missing values.


,Missing Values,% of Total Values
id_24,585793,99.2
id_25,585408,99.1
id_07,585385,99.1
id_08,585385,99.1
id_21,585381,99.1
id_26,585377,99.1
id_27,585371,99.1
id_23,585371,99.1
id_22,585371,99.1
dist2,552913,93.6


In [16]:
# Number of each type of column
train.dtypes.value_counts()

float16    354
float32     45
object      31
int32        2
int16        1
int8         1
dtype: int64

In [17]:
# Number of unique classes (categories) in each object column
train.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

ProductCD           5
card4               4
card6               4
P_emaildomain      59
R_emaildomain      60
M1                  2
M2                  2
M3                  2
M4                  3
M5                  2
M6                  2
M7                  2
M8                  2
M9                  2
id_12               2
id_15               3
id_16               2
id_23               3
id_27               2
id_28               2
id_29               2
id_30              75
id_31             130
id_33             260
id_34               4
id_35               2
id_36               2
id_37               2
id_38               2
DeviceType          2
DeviceInfo       1786
dtype: int64

## Categorical Encoding

- Label Encoding
- One Hot Encoding
- Mean Encoding (different strategies to implement it) ... To see more, https://www.youtube.com/watch?v=AV_hJN1ALnI&list=PLpQWTe-45nxL3bhyAJMEs90KF_gZmuqtm&index=37